In [ ]:
from census import Census
from us import states
import pandas as pd
import censusdata
import numpy as np
import csv
import seaborn as sns

Input information about groups, outcomes, year, state,etc

In [ ]:
NGroups = input('Enter number of groups: ')
NOutcomes = input('Enter number of outcomes: ')
Yr = input('Enter year:')
Statecode = input('Enter state code : ')
ACS = input ('Enter acs5 or acs1 : ')

In [ ]:
#Converting strings into integers
NGroups = int(NGroups)
NOutcomes = int(NOutcomes)
Yr = int(Yr)

Open the CensusVariable.csv and Countycode.txt file   
If you are performing this analysis for more than once, you should change the file names

In [ ]:
with open('CensusVariable2.csv', 'r') as f:
    reader = csv.reader(f)
    Your_List = list(reader)
    
#Create a list of variable names
CensusVariableList = [val for sublist in Your_List for val in sublist]
print ('Census Variable List\n')
print (CensusVariableList)
print ('-'*50)

file = open('Countycode.txt', 'r') 
for line in file:
    CountyCode = line   
    
print ('Country Code List\n')
print(CountyCode)


Download ACS data for different group categories  
Change column names  
Add row which is the sum of the table  
Append the row into a masterfile  

Repeat the above steps using a while loop for all the groups  

In [ ]:
i=0
Analysis_Step_1 = pd.DataFrame(columns=[])
while i < NGroups:
    
    Group1 = censusdata.download(ACS, Yr,
                             censusdata.censusgeo([('state', Statecode), ('county', CountyCode)]),
                             CensusVariableList[NOutcomes*i:NOutcomes*(i+1)])
    Group1.columns = np.arange(len(Group1.columns))
    Column_name = 'Group' + str(i+1)
    Group1.loc[Column_name] = Group1.sum()
    Analysis_Step_1 = Analysis_Step_1.append(Group1.loc[Column_name])
    i = i+1
    
Analysis_Step_1
    
    

Transpose the matrix

In [ ]:
Analysis_Step_2 = Analysis_Step_1.transpose()
Analysis_Step_2

Calculate ratio  

'lambda' function - https://dbader.org/blog/python-lambda-functions

In [ ]:
Analysis_Step_3 = Analysis_Step_2.apply(lambda x: x/x.sum(), axis=1)
Analysis_Step_3.style.set_precision(2)

Calculate deviation

In [ ]:
Analysis_Step_4 =  Analysis_Step_3 - Analysis_Step_3.iloc[0]
Analysis_Step_4
Analysis_Step_4.style.set_precision(2) #Sets the precision of numbers

Calculate Skew

In [ ]:
cm = sns.light_palette("green", as_cmap=True)
Analysis_Step_5 = abs(Analysis_Step_4.multiply((Analysis_Step_2.sum(axis=1)/Analysis_Step_2.iloc[0].sum()), axis=0))
Skew = Analysis_Step_5.values.sum()/2
print ("Skew : ", Skew)
Analysis_Step_5 = Analysis_Step_5.style.background_gradient(cmap=cm)   #to colour the table in gradients
Analysis_Step_5